The code below imports the necessary libraries for this project.

In [1]:
!pip install pandas numpy scikit-learn transformers torch xgboost imblearn tqdm

import pandas as pd
import numpy as np
import re
import random
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset

from transformers import BertTokenizer, BertModel

from imblearn.over_sampling import SMOTE

The code below loads the data and cleans it.

In [2]:
# Generate random seeds for reporductibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

# Load the data
trainDf = pd.read_csv("train_E6oV3lV.csv")
testDf = pd.read_csv("test_tweets_anuFYb8.csv")

# Function for cleaning the data
def cleanTweet(tweet):
    tweet = tweet.lower()
    tweet = tweet.replace("@user", "")
    tweet = tweet.replace("#", "")
    tweet = re.sub(r"http\S+", "", tweet)
    tweet = re.sub(r"[^a-z0-9\s]", "", tweet)
    tweet = re.sub(r"\s+", " ", tweet).strip()
    return tweet

# Clean the data
trainDf["cleanTweet"] = trainDf["tweet"].apply(cleanTweet)
testDf["cleanTweet"] = testDf["tweet"].apply(cleanTweet)

The code below implements the BERT form of featurization. We use BERT emebeddings since it understand contextual meaning, informal language and slang usage more efficiently than TF-IDF Vectorization. 

In [3]:
# Use MPS for faster embeddings generated
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased").to(device)
model.eval()

# Dataset class for BERT embeddings
class TweetDataset(Dataset):
    def __init__(self, texts, tokenizer, maxLen = 64):
        self.texts = texts
        self.tokenizer = tokenizer
        self.maxLen = maxLen

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation = True,
            padding = "max_length",
            max_length = self.maxLen,
            return_tensors = "pt"
        )
        return {key: val.squeeze(0) for key, val in encoding.items()}

def generateEmbeddings(tweets, tokenizer, model, device, path = "BERTEmbeddings.pt"):
    print("Generating embeddings...")

    # Instantiate dataset and dataloader
    dataset = TweetDataset(tweets, tokenizer)
    loader = DataLoader(dataset, batch_size = 16)
                        
    # Collect embeddings
    allEmbeddings = []

    # Generate [CLS] emebeddings batch-by-batch and store them
    with torch.no_grad():
        for batch in tqdm(loader, desc = "Generating BERT Embeddings"):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            clsEmbeddings = outputs.last_hidden_state[:, 0, :].cpu()
            allEmbeddings.append(clsEmbeddings)

    # Concatenate to final matrix
    Xbert = torch.cat(allEmbeddings)
    print(f"Shape of Final Embedding Matrix: {Xbert.shape}")

    # Save to cache
    torch.save(Xbert, path)
    print(f"Embeddings saved to {path}")

    return Xbert.cpu().numpy()

# Generate embeddings for train set
tweets = trainDf["cleanTweet"].tolist()
Xbert = generateEmbeddings(tweets, tokenizer, model, device)


Generating embeddings...


Generating BERT Embeddings: 100%|██████████| 1998/1998 [07:51<00:00,  4.24it/s]


Shape of Final Embedding Matrix: torch.Size([31962, 768])
Embeddings saved to BERTEmbeddings.pt


The code below implements the Multilayer Perceptron (MLP) deep learning model.

In [4]:
# Prepare target labels
ybert = trainDf["label"].values

# Split train/validation
XTrain, XVal, yTrain, yVal = train_test_split(Xbert, ybert, test_size = 0.2, random_state = 42)

# Balance training set skewed towards non-hate speech labels
smote = SMOTE(random_state = 42, k_neighbors = 3)
XTrainSMOTE, yTrainSMOTE = smote.fit_resample(XTrain, yTrain)

# Change device to CPU since MPS freezes on last few batches
device = torch.device("cpu")

# Create a Enhanced MLP Deep Learning Model
class EnhancedMLP(nn.Module):
    def __init__(self, inputDim = 768, hiddenDims = [512, 256, 128], numClasses = 2, dropoutRate = 0.4):
        super(EnhancedMLP, self).__init__()

        # Build multiple hidden layers
        layers = []
        prevDim = inputDim

        for i, hiddenDim in enumerate(hiddenDims):
            layers.append(nn.Linear(prevDim, hiddenDim))
            layers.append(nn.BatchNorm1d(hiddenDim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropoutRate))
            prevDim = hiddenDim

        # Output layer
        layers.append(nn.Linear(prevDim, numClasses))

        self.network = nn.Sequential(*layers)

        # Initialize weights
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        return self.network(x)

# Train the MLP Model
def trainEnhancedMLP(XTrainSMOTE, yTrainSMOTE, XVal, yVal):
    # Set device to CPU, as MPS freezes at last couple of batches
    device = torch.device("cpu")

    # Convert to tensors
    XTrainTensor = torch.from_numpy(XTrainSMOTE.astype(np.float32))
    yTrainTensor = torch.from_numpy(yTrainSMOTE.astype(np.int64))
    XValTensor = torch.from_numpy(XVal.astype(np.float32))
    yValTensor = torch.from_numpy(yVal.astype(np.int64))

    # Create data loaders
    trainDataset = TensorDataset(XTrainTensor, yTrainTensor)
    trainLoader = DataLoader(trainDataset, batch_size = 64, shuffle = True, num_workers = 0, pin_memory = False)

    # Initialize MLP Deep Learning Model
    model = EnhancedMLP(
        inputDim = 768,
        hiddenDims = [512, 256, 128],
        numClasses = 2,
        dropoutRate = 0.4
    ).to(device)

    # Use weighted loss to handle class imbalance
    classWeights = compute_class_weight("balanced", classes = np.unique(yTrainSMOTE), y = yTrainSMOTE)
    classWeightsTensor = torch.tensor(classWeights, dtype = torch.float32).to(device)

    # Use weighted CrossEntropyLoss
    criterion = nn.CrossEntropyLoss(weight = classWeightsTensor)

    # Use AdamW optimizer with weight decay
    optimizer = optim.AdamW(model.parameters(), lr = 0.001, weight_decay = 0.01)

    # Learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode = "max", factor = 0.5, patience = 3)

    # Training parameters
    epochs = 20
    bestHateF1 = 0.0
    bestModelState = None
    patience = 5
    patienceCounter = 0

    print("Training Enhanced MLP Deep Learning Model...")

    # Training Phase with Early Stopping
    for epoch in range(epochs):
        model.train()
        totalLoss = 0.0

        for batchX, batchY in tqdm(trainLoader, desc = f"Epoch {epoch + 1}/{epochs}"):
            batchX, batchY = batchX.to(device), batchY.to(device)

            optimizer.zero_grad()
            outputs = model(batchX)
            loss = criterion(outputs, batchY)
            loss.backward()

            # Gradient clipping to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = 1.0)

            optimizer.step()
            totalLoss += loss.item()
    
        # Validation Phase
        model.eval()
        with torch.no_grad():
            valOutputs = model(XValTensor.to(device))
            valProbs = torch.softmax(valOutputs, dim = 1)

            # Use optimal threshold for hate speech detection, lower threshold favors hate speech detection
            threshold = 0.3
            valPreds = (valProbs[:, 1] > threshold).long()
            valPredsNP = valPreds.cpu().numpy()

        # Calculate F1 Scores
        F1Macro = f1_score(yVal, valPredsNP, average = "macro")
        F1Hate = f1_score(yVal, valPredsNP, pos_label = 1)
        F1NonHate = f1_score(yVal, valPredsNP, pos_label = 0)
        accuracy = (valPredsNP == yVal).mean()

        print(f"Epoch {epoch + 1}: Loss = {totalLoss/len(trainLoader):.3f}, "
            f"Accuracy = {accuracy:.4f}, F1Hate = {F1Hate:.4f}, F1NonHate = {F1NonHate:.4f}, "
            f"F1Macro = {F1Macro:.4f}")
        
        # Update learning rate scheduler
        scheduler.step(F1Hate)

        # Track best model based on hate F1 score
        if F1Hate > bestHateF1:
            bestHateF1 = F1Hate
            bestHateF1Epoch = epoch
            bestModelState = model.state_dict().copy()
            patienceCounter = 0
        else:
            patienceCounter += 1

        if patienceCounter >= patience:
            print(f"Early stopping at Epoch {epoch + 1}")
            break

    # Load best model for final evaluation
    if bestModelState is not None:
        model.load_state_dict(bestModelState)
        print(f"\nLoaded Best Model with Hate F1 Score: {bestHateF1:.4f} at Epoch {bestHateF1Epoch + 1}")

    # Final Evaluation with Threshold Tuning
    model.eval()
    with torch.no_grad():
        finalOutputs = model(XValTensor.to(device))
        finalProbs = torch.softmax(finalOutputs, dim = 1)

        # Test different thresholds to find optimal one
        thresholds = np.arange(0.1, 0.9, 0.05)
        bestThreshold = 0.5
        bestF1 = 0.0

        for threshold in thresholds:
            preds = (finalProbs[:, 1] > threshold).long().cpu().numpy()
            f1 = f1_score(yVal, preds, pos_label = 1)
            if f1 > bestF1:
                bestF1 = f1
                bestThreshold = threshold
        
        print(f"Optimal Threshold for Hate Speech Detection: {bestThreshold:.3f}")

        # Final prediction using optimal threshold
        finalPreds = (finalProbs[:, 1] > bestThreshold).long()
        finalPredsNP = finalPreds.cpu().numpy()

    # Print results
    print(f"\nFinal Results with Optimized Threshold\n")

    finalAccuracy = (finalPredsNP == yVal).mean()
    print(f"Final Accuracy: {finalAccuracy:.4f}")

    print("Final Classification Report:")
    print(classification_report(yVal, finalPredsNP, target_names = ["Non-Hate", "Hate"]))

    print("Final Confusion Matrix:")
    print(confusion_matrix(yVal, finalPredsNP))

    F1HateFinal = f1_score(yVal, finalPredsNP, pos_label = 1)
    F1NonHateFinal = f1_score(yVal, finalPredsNP, pos_label = 0)
    F1MacroFinal = f1_score(yVal, finalPredsNP, average = "macro")

    print(f"\nFinal F1 Scores:")
    print(f"    Hate Speech F1: {F1HateFinal:.4f}")
    print(f"    Non-Hate Speech F1: {F1NonHateFinal:.4f}")
    print(f"    Macro F1: {F1MacroFinal:.4f}")

    # Save the best model
    modelSavePath = f"enhancedMLPModel.pth"
    torch.save({
        "model_state_dict": model.state_dict(),
        "model_architecture": {
            "inputDim": 768,
            "hiddenDims": [512, 256, 128],
            "numClasses": 2,
            "dropoutRate": 0.4
        },
        "best_hate_f1": bestHateF1,
        "optimal_threshold": bestThreshold,
        "final_metrics": {
            "accuracy": finalAccuracy,
            "hate_f1": F1HateFinal,
            "non_hate_f1": F1NonHateFinal,
            "macro_f1": F1MacroFinal
        }
    }, modelSavePath)

    print(f"\nBest Model saved to: {modelSavePath}")
    print(f"To load this model later, use:")
    print(f"    checkpoint = torch.load(\"{modelSavePath}\")")
    print(f"    model = MLP(**checkpoint['model_architecture'])")
    print(f"    model.load_state_dict(checkpoint['model_state_dict'])")

    return model, finalPredsNP, bestThreshold

print("Enhanced MLP Deep Learning Model - SMOTE Balanced\n")
enhancedModel, enhancedPreds, optimalThreshold = trainEnhancedMLP(XTrainSMOTE, yTrainSMOTE, XVal, yVal)

Enhanced MLP Deep Learning Model - SMOTE Balanced

Training Enhanced MLP Deep Learning Model...


Epoch 1/20: 100%|██████████| 744/744 [00:08<00:00, 86.15it/s] 


Epoch 1: Loss = 0.248, Accuracy = 0.8167, F1Hate = 0.4099, F1NonHate = 0.8915, F1Macro = 0.6507


Epoch 2/20: 100%|██████████| 744/744 [00:04<00:00, 161.21it/s]


Epoch 2: Loss = 0.145, Accuracy = 0.8911, F1Hate = 0.5146, F1NonHate = 0.9387, F1Macro = 0.7267


...

Epoch 13: Loss = 0.026, Accuracy = 0.9499, F1Hate = 0.6322, F1NonHate = 0.9731, F1Macro = 0.8027


Epoch 14/20: 100%|██████████| 744/744 [00:04<00:00, 182.47it/s]


Epoch 14: Loss = 0.027, Accuracy = 0.9402, F1Hate = 0.6134, F1NonHate = 0.9676, F1Macro = 0.7905


Epoch 15/20: 100%|██████████| 744/744 [00:04<00:00, 178.27it/s]


Epoch 15: Loss = 0.015, Accuracy = 0.9493, F1Hate = 0.6400, F1NonHate = 0.9727, F1Macro = 0.8064
Early stopping at Epoch 15

Loaded Best Model with Hate F1 Score: 0.6452 at Epoch 10
Optimal Threshold for Hate Speech Detection: 0.200

Final Results with Optimized Threshold

Final Accuracy: 0.9476
Final Classification Report:
              precision    recall  f1-score   support

    Non-Hate       0.97      0.97      0.97      5937
        Hate       0.63      0.65      0.64       456

    accuracy                           0.95      6393
   macro avg       0.80      0.81      0.81      6393
weighted avg       0.95      0.95      0.95      6393

Final Confusion Matrix:
[[5760  177]
 [ 158  298]]

Final F1 Scores:
    Hate Speech F1: 0.6402
    Non-Hate Speech F1: 0.9717
    Macro F1: 0.8060

Best Model saved to: enhancedMLPModel.pth
To load this model later, use:
    checkpoint = torch.load("enhancedMLPModel.pth")
    model = MLP(**checkpoint['model_architecture'])
    model.load_state_

The code below loads the model and tests it on the testing file.

In [5]:
# Generate embeddings for test set
testTweets = testDf["cleanTweet"].tolist()
XTestBert = generateEmbeddings(testTweets, tokenizer, model.to(device), device, "testBERTEmbeddings.pt")

# Load the trained model
checkpoint = torch.load("enhancedMLPModel.pth")
testModel = EnhancedMLP(**checkpoint["model_architecture"])
testModel.load_state_dict(checkpoint["model_state_dict"])
testModel.eval()

# Get optimal threshold from saved model
optimalThreshold = checkpoint["optimal_threshold"]

# Make predictions on test set
device = torch.device("cpu")
testModel.to(device)

XTestTensor = torch.from_numpy(XTestBert.astype(np.float32)).to(device)

with torch.no_grad():
    testOutputs = testModel(XTestTensor)
    testProbs = torch.softmax(testOutputs, dim = 1)
    testPreds = (testProbs[:, 1] > optimalThreshold).long().cpu().numpy()

# Create predictions file
testDf["label"] = testPreds
predictions = testDf[["id", "label"]].copy()
predictions.to_csv("testPredictions.csv", index = False)

print(f"Test predictions completed!")
print(f"Predictions saved to 'testPredictions.csv'")
print(f"Total predictions: {len(testPreds)}")
print(f"Hate speech predictions: {sum(testPreds)}")
print(f"Non-hate speech predictions: {len(testPreds) - sum(testPreds)}")

Generating embeddings...


Generating BERT Embeddings: 100%|██████████| 1075/1075 [12:55<00:00,  1.39it/s]


Shape of Final Embedding Matrix: torch.Size([17197, 768])
Embeddings saved to testBERTEmbeddings.pt


/var/folders/yk/6b7wzf7x14x25j715qn6rth00000gn/T/ipykernel_67593/1142247259.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("enhancedMLPModel.pth

Test predictions completed!
Predictions saved to 'testPredictions.csv'
Total predictions: 17197
Hate speech predictions: 1329
Non-hate speech predictions: 15868


The code below provides a function for the trained model to predict on user inputs.

In [6]:
# Function to predict text hate speech
def predictHateSpeech(text, threshold = None):
    # Load model and get optimal threshold
    checkpoint = torch.load("enhancedMLPModel.pth")
    model = EnhancedMLP(**checkpoint["model_architecture"])
    model.load_state_dict(checkpoint["model_state_dict"])
    model.eval()
    
    if threshold is None:
        threshold = checkpoint["optimal_threshold"]
    
    # Clean the input text
    cleanedText = cleanTweet(text)
    
    # Generate BERT embedding for the text
    device = torch.device("cpu")
    model.to(device)
    
    # Load BERT model for embedding
    bertModel = BertModel.from_pretrained("bert-base-uncased")
    bertModel.eval().to(device)
    
    # Tokenize and encode
    encoding = tokenizer(
        cleanedText,
        truncation = True,
        padding = "max_length",
        max_length = 64,
        return_tensors = "pt"
    )
    
    # Generate embedding
    with torch.no_grad():
        outputs = bertModel(**encoding.to(device))
        embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()
    
    # Make prediction
    embeddingTensor = torch.from_numpy(embedding.astype(np.float32)).to(device)
    
    with torch.no_grad():
        output = model(embeddingTensor)
        probs = torch.softmax(output, dim=1)
        hateProb = probs[0, 1].item()
        prediction = 1 if hateProb > threshold else 0
    
    # Prepare result
    result = {
        "original_text": text,
        "cleaned_text": cleanedText,
        "prediction": prediction,
        "label": "Hate Speech" if prediction == 1 else "Non-Hate Speech",
        "confidence": hateProb,
        "threshold_used": threshold
    }
    
    return result

# Example Non-Hate Speech Prediction
result = predictHateSpeech("I love your approach to life, it's very unique.")
print("\nNon-Hate Speech Statement: ")
for row in result:
    print(f"{row}: {result[row]}")

# Example Hate Speech Prediction
result = predictHateSpeech("You are such a stupid idiot.")
print("\nHate Speech Statement: ")
for row in result:
    print(f"{row}: {result[row]}")

/var/folders/yk/6b7wzf7x14x25j715qn6rth00000gn/T/ipykernel_67593/799524318.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("enhancedMLPModel.pth"


Non-Hate Speech Statement: 
original_text: I love your approach to life, it's very unique.
cleaned_text: i love your approach to life its very unique
prediction: 0
label: Non-Hate Speech
confidence: 1.3302735624165507e-06
threshold_used: 0.20000000000000004

Hate Speech Statement: 
original_text: You are such a stupid idiot.
cleaned_text: you are such a stupid idiot
prediction: 1
label: Hate Speech
confidence: 0.997912585735321
threshold_used: 0.20000000000000004
